###  Install

 - Open a terminal and install `cuquantum-python` and `pennylane-lightning-qpu`:
 
   ```
   conda install -c conda-forge cuquantum-python
   pip install pennylane-lightning-qpu
   ```

In [35]:
from pennylane import numpy as np
import pennylane as qml
from pennylane.pauli import pauli_mult, pauli_mult_with_phase, pauli_word_to_string, string_to_pauli_word
from scipy.linalg import eigh
from scipy.linalg import det

In [2]:
def angstrom_to_bohr(dist):
    return dist*1.88973

In [3]:
n_electrons = 4
n_orbitals = 6

symbols = ["H", "Be", "H"]
coordinates = angstrom_to_bohr(np.array([0.0, 0.0, -1.291, 0.0, 0.0,0.0, 0.0, 0.0, 1.291]))
#coordinates = angstrom_to_bohr(np.array([0.0, 0.0, -1.33, 0.0, 0.0,0.0, 0.0, 0.0, 1.33]))


H, n_qubits = qml.qchem.molecular_hamiltonian(symbols,
                                                coordinates,
                                                active_electrons=n_electrons,
                                                active_orbitals=n_orbitals,
                                                grouping_method='qwc')
print("Number of qubits = ", n_qubits)
# print("The Hamiltonian is ", H)

Number of qubits =  12


In [4]:
print(len(H.coeffs))

327


In [5]:
generators = qml.symmetry_generators(H)
paulixops = qml.paulix_ops(generators, n_qubits)
paulix_sector = qml.qchem.optimal_sector(H, generators, n_electrons)

/opt/conda/lib/python3.10/site-packages/pennylane/operation.py:1830: UserWarning: Tensor object acts on overlapping wires; in some PennyLane functions this will lead to undefined behaviour
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/pennylane/operation.py:1993: UserWarning: Tensor object acts on overlapping wires; in some PennyLane functions this will lead to undefined behaviour
  warnings.warn(


In [6]:
H_tapered = qml.taper(H, generators, paulixops, paulix_sector)
# print(H_tapered)

In [7]:
print(len(H_tapered.ops))

268


In [8]:
state_tapered = qml.qchem.taper_hf(generators, paulixops, paulix_sector,
                                   num_electrons=n_electrons, num_wires=len(H.wires))
print(state_tapered)

[1 1 1 1 0 0 0]


In [9]:
len(H.wires)

12

In [10]:
dev = qml.device("default.qubit", wires=H_tapered.wires)
@qml.qnode(dev)
def circuit():
    qml.BasisState(np.array(state_tapered), wires=H_tapered.wires)
    return qml.state()

In [11]:
qubit_state = circuit()
HF_energy = qubit_state.T @ qml.utils.sparse_hamiltonian(H_tapered).toarray() @ qubit_state
print(f"HF energy (tapered): {np.real(HF_energy):.8f} Ha")

HF energy (tapered): -15.56135260 Ha


In [12]:
singles, doubles = qml.qchem.excitations(n_electrons, len(H.wires))
print('singles', len(singles), 'doubles ', len(doubles))
tapered_doubles = [
    qml.taper_operation(qml.DoubleExcitation, generators, paulixops, paulix_sector,
                        wire_order=H.wires, op_wires=double) for double in doubles
]
tapered_singles = [
    qml.taper_operation(qml.SingleExcitation, generators, paulixops, paulix_sector,
                        wire_order=H.wires, op_wires=single) for single in singles
]

singles 16 doubles  76


In [13]:
dev = qml.device("default.qubit", wires=H_tapered.wires)
@qml.qnode(dev)
def tapered_circuit(params):
    qml.BasisState(state_tapered, wires=H_tapered.wires)
    for idx, tapered_op in enumerate(tapered_doubles + tapered_singles):
        tapered_op(params[idx])
    return qml.expval(H_tapered)

In [14]:
optimizer = qml.GradientDescentOptimizer(stepsize=0.5)
params = np.zeros(len(doubles) + len(singles), requires_grad=True)

In [15]:
tapered_circuit(params)

tensor(-15.5613526, requires_grad=True)

In [16]:
for n in range(41):
    params, energy = optimizer.step_and_cost(tapered_circuit, params)
    if n % 10 == 0:
        print(f"n: {n}, E: {energy:.8f} Ha")

n: 0, E: -15.56135260 Ha
n: 10, E: -15.59398767 Ha
n: 20, E: -15.59407853 Ha
n: 30, E: -15.59408220 Ha
n: 40, E: -15.59408238 Ha


In [18]:
@qml.qnode(dev)
def tapered_state(params):
    qml.BasisState(state_tapered, wires=H_tapered.wires)
    for idx, tapered_op in enumerate(tapered_doubles + tapered_singles):
        tapered_op(params[idx])
    return qml.state()

In [19]:
input_state = tapered_state(params)

In [20]:
# Return the Pauli words of H in string form
def extract_basis_str(H, wire_map):
    return set(pauli_word_to_string(op, wire_map = wire_map) for op in H.ops)

In [21]:
# Generate the cummulative space CSK
def cummulative_space(H, K, wire_map):
    
    CSK = set()
    if K == 0:
        Id = 'I'
        for i in range(qubits-1):
            Id += 'I'
        return {Id}
    
    elif K==1:
        return extract_basis_str(H, wire_map)
    
    else:
        CS1 = extract_basis_str(H, wire_map)
        
        Id = 'I'
        for i in range(qubits-1):
            Id += 'I'
            
        basis = [{''.join(['I' for _ in range(qubits_tap)])}, CS1]
        
        for k in range(K-1):
            CSK.update(basis[-2])
            basis.append(increment_cummulative_space( basis[-1].difference(a)  ,CS1))
            
        CSK.update(*basis[-2:])
    return CSK

In [22]:
# Help generate the cummulative space
def increment_cummulative_space(basis, CS1):
    return set(pauli_word_to_string(pauli_mult(s1, s2)) for s1 in basis for s2 in CS1)


### Molecule

In [25]:
symbols = ["H","Be","H"]
dist = 1.291
geometry = angstrom_to_bohr(np.array([[0, 0, -dist], [0., 0, 0], [0, 0, dist]]))
mol = qml.qchem.Molecule(symbols, geometry)

In [26]:
active_electrons = 4
active_orbitals = 6

H, qubits = qml.qchem.molecular_hamiltonian(mol.symbols,
                                            mol.coordinates,
                                            active_electrons=active_electrons,
                                            active_orbitals=active_orbitals,
                                            grouping_method = 'qwc',
                                            args=[mol.coordinates])

### Tapering

In [27]:
generators = qml.symmetry_generators(H)
paulixops = qml.paulix_ops(generators, qubits)
paulix_sector = qml.qchem.optimal_sector(H, generators, active_electrons)

H_tapered = qml.taper(H, generators, paulixops, paulix_sector)

In [28]:
qubits_tap = qubits-len(paulixops)
wire_tapered = [ paulixops[i].wires[0] for i in range(len(paulixops))]
wire_tap = [ x for x in range(qubits) if not x in wire_tapered ]
wire_map = {wire:i for i,wire in enumerate(wire_tap)}

In [30]:
dev = qml.device("lightning.gpu", wires=qubits_tap)
@qml.qnode(dev)
def expect(P, input_state):
    qml.QubitStateVector(input_state, range(qubits_tap))
            
    return qml.expval(P)


def expectation_value(input_state, P1, P2, P3=None):    
    if P3 is None:
        P, phase = pauli_mult_with_phase(P1, P2)
    else:
        P_, phase_ = pauli_mult_with_phase(P1, P3)
        P, phase = pauli_mult_with_phase(P_ ,P2)
        phase += phase_

    return np.exp(1j * np.pi * phase / 2) * expect(P, input_state)

In [31]:
# Construct the overlap matrix S
def S_mat(input_state, basis):
    N = len(basis)
    S_mat = np.ones((N, N), dtype=complex)
    for i in range(N):
        for j in range(i):
            S_mat[i,j] = expectation_value(input_state, 
                                           string_to_pauli_word(basis[i]),
                                           string_to_pauli_word(basis[j]))
            S_mat[j,i] = S_mat[i,j].conjugate()
    return S_mat

In [72]:
# Construct the matrix D
def D_mat(input_state, basis, H):
    N = len(basis)
    D_mat = np.zeros((N, N), dtype=complex)

    for i in range(N):
        print(i)
        for j in range(i):
            D_mat[i, j] = sum(H.coeffs[k] * expectation_value(input_state, string_to_pauli_word(basis[i]),
                                                              string_to_pauli_word(basis[j]), H.ops[k])
                              for k in range(len(H.coeffs)))
            D_mat[j, i] = D_mat[i,j].conjugate()
            
        D_mat[i, i] = sum(H.coeffs[k] * expectation_value(input_state, string_to_pauli_word(basis[i]),
                                                          string_to_pauli_word(basis[i]), H.ops[k])
                          for k in range(len(H.coeffs))).real
    return D_mat

### Generalized eigenvalue problem

In [81]:
# pauli_word_to_string(H_tapered.ops[4], wire_map)
qubits_tap

7

In [78]:
s = 0
for k in range(len(H_tapered.coeffs)):
    print(k)
    s += H_tapered.coeffs[k] * expectation_value(input_state, string_to_pauli_word(basis[1]), string_to_pauli_word(basis[1]), H_tapered.ops[k])
s

0
1
2
3
4


WireError: Wire with label 8 not found in <Wires = [0, 1, 2, 3, 4, 5, 6]>.

In [70]:
basis = ['IIIIIII','XIIIIII','IXIIIII','IIXIIII','IIIXIII','IIIIXII','IIIIIXI','IIIIIIX']

S = S_mat(input_state, basis)
print(det(S))

(0.9749389319140243-6.672510373622864e-21j)


In [73]:
D = D_mat(input_state, basis, H_tapered)

0


WireError: Wire with label 8 not found in <Wires = [0, 1, 2, 3, 4, 5, 6]>.

In [ ]:
vals, vect = eigh(D,S, eigvals_only=False)
print(vals)

In [ ]:
from matplotlib import pyplot as plt


plt.plot(abs(vect[0])**2,'.')
plt.yscale('log')